# Введение в MapReduce модель на Python


In [ ]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

In [ ]:
def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

Модель элемента данных

In [ ]:
class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

In [ ]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

Функция RECORDREADER моделирует чтение элементов с диска или по сети.

In [ ]:
def RECORDREADER():
  return [(u.id, u) for u in input_collection]

In [ ]:
list(RECORDREADER())

[(0, User(id=0, age=55, social_contacts=20, gender='male')),
 (1, User(id=1, age=25, social_contacts=240, gender='female')),
 (2, User(id=2, age=25, social_contacts=500, gender='female')),
 (3, User(id=3, age=33, social_contacts=800, gender='female'))]

In [ ]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

In [ ]:
map_output = flatten(map(lambda x: MAP(*x), RECORDREADER()))
map_output = list(map_output) # materialize
map_output

[(25, User(id=1, age=25, social_contacts=240, gender='female')),
 (25, User(id=2, age=25, social_contacts=500, gender='female')),
 (33, User(id=3, age=33, social_contacts=800, gender='female'))]

In [ ]:
def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

In [ ]:
shuffle_output = groupbykey(map_output)
shuffle_output = list(shuffle_output)
shuffle_output

[(25,
  [User(id=1, age=25, social_contacts=240, gender='female'),
   User(id=2, age=25, social_contacts=500, gender='female')]),
 (33, [User(id=3, age=33, social_contacts=800, gender='female')])]

In [ ]:
reduce_output = flatten(map(lambda x: REDUCE(*x), shuffle_output))
reduce_output = list(reduce_output)
reduce_output

[(25, 370.0), (33, 800.0)]

Все действия одним конвейером!

In [ ]:
list(flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))))))

[(25, 370.0), (33, 800.0)]

# **MapReduce**
Выделим общую для всех пользователей часть системы в отдельную функцию высшего порядка. Это наиболее простая модель MapReduce, без учёта распределённого хранения данных.

Пользователь для решения своей задачи реализует RECORDREADER, MAP, REDUCE.

In [ ]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

## Спецификация MapReduce



```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

mapreduce ((k1,v1)*) -> (k3,v3)*
groupby ((k2,v2)*) -> (k2,v2*)*
flatten (e2**) -> e2*

mapreduce .map(f).flatten.groupby(k2).map(g).flatten
```




# Примеры

## SQL

In [ ]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

def RECORDREADER():
  return [(u.id, u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(25, 370.0), (33, 800.0)]

## Matrix-Vector multiplication

In [ ]:
from typing import Iterator
import numpy as np

mat = np.ones((5,4))
vec = np.random.rand(4) # in-memory vector in all map tasks

def MAP(coordinates:(int, int), value:int):
  i, j = coordinates
  yield (i, value*vec[j])

def REDUCE(i:int, products:Iterator[NamedTuple]):
  sum = 0
  for p in products:
    sum += p
  yield (i, sum)

def RECORDREADER():
  for i in range(mat.shape[0]):
    for j in range(mat.shape[1]):
      yield ((i, j), mat[i,j])

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, 2.905589809636405),
 (1, 2.905589809636405),
 (2, 2.905589809636405),
 (3, 2.905589809636405),
 (4, 2.905589809636405)]

## Inverted index

In [ ]:
from typing import Iterator

d1 = "it is what it is"
d2 = "what is it"
d3 = "it is a banana"
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    yield ("{}".format(docid), document)

def MAP(docId:str, body:str):
  for word in set(body.split(' ')):
    yield (word, docId)

def REDUCE(word:str, docIds:Iterator[str]):
  yield (word, sorted(docIds))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('what', ['0', '1']),
 ('is', ['0', '1', '2']),
 ('it', ['0', '1', '2']),
 ('a', ['2']),
 ('banana', ['2'])]

## WordCount

In [ ]:
from typing import Iterator

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    for (lineid, line) in enumerate(document.split('\n')):
      yield ("{}:{}".format(docid,lineid), line)

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('', 3), ('it', 9), ('is', 9), ('what', 5), ('a', 1), ('banana', 1)]

# MapReduce Distributed

Добавляется в модель фабрика RECORDREARER-ов --- INPUTFORMAT, функция распределения промежуточных результатов по партициям PARTITIONER, и функция COMBINER для частичной аггрегации промежуточных результатов до распределения по новым партициям.

In [ ]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

## Спецификация MapReduce Distributed


```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

e1 (k1, v1)
e2 (k2, v2)
partition1 (k2, v2)*
partition2 (k2, v2*)*

flatmap (e1->e2*, e1*) -> partition1*
groupby (partition1*) -> partition2*

mapreduce ((k1,v1)*) -> (k3,v3)*
mapreduce .flatmap(f).groupby(k2).flatmap(g)
```



## WordCount

In [ ]:
from typing import Iterator
import numpy as np

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for (docid, document) in enumerate(split):
      for (lineid, line) in enumerate(document.split('\n')):
        yield ("{}:{}".format(docid,lineid), line)

  split_size =  int(np.ceil(len(documents)/maps))
  for i in range(0, len(documents), split_size):
    yield RECORDREADER(documents[i:i+split_size])

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

# try to set COMBINER=REDUCER and look at the number of values sent over the network
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

56 key-value pairs were sent over a network.


[(0, [('', 6), ('a', 2), ('banana', 2), ('is', 18), ('it', 18), ('what', 10)]),
 (1, [])]

## TeraSort

In [ ]:
import numpy as np

input_values = np.random.rand(30)
maps = 3
reducers = 2
min_value = 0.0
max_value = 1.0

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for value in split:
        yield (value, None)

  split_size =  int(np.ceil(len(input_values)/maps))
  for i in range(0, len(input_values), split_size):
    yield RECORDREADER(input_values[i:i+split_size])

def MAP(value:int, _):
  yield (value, None)

def PARTITIONER(key):
  global reducers
  global max_value
  global min_value
  bucket_size = (max_value-min_value)/reducers
  bucket_id = 0
  while((key>(bucket_id+1)*bucket_size) and ((bucket_id+1)*bucket_size<max_value)):
    bucket_id += 1
  return bucket_id

def REDUCE(value:int, _):
  yield (None,value)

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None, PARTITIONER=PARTITIONER)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

30 key-value pairs were sent over a network.


[(0,
  [(None, 0.0059671639991895065),
   (None, 0.07724245496172),
   (None, 0.08440804135613444),
   (None, 0.13575647907181598),
   (None, 0.14404826813474803),
   (None, 0.21204275967955666),
   (None, 0.21869633101751806),
   (None, 0.25055756276216923),
   (None, 0.28642389538931257),
   (None, 0.3834487515438496),
   (None, 0.3913614390023946),
   (None, 0.4041378102237341),
   (None, 0.41854626274930695),
   (None, 0.4704310153549396),
   (None, 0.4776995227348928),
   (None, 0.48992216726013693)]),
 (1,
  [(None, 0.5005353544023048),
   (None, 0.5135664686748047),
   (None, 0.53391984417089),
   (None, 0.5587932025401512),
   (None, 0.5673804905854288),
   (None, 0.6926646597910275),
   (None, 0.7237444251339501),
   (None, 0.7557883138083207),
   (None, 0.785709769245918),
   (None, 0.7937098630029404),
   (None, 0.7942646850708935),
   (None, 0.9160468126494941),
   (None, 0.9618068292060864),
   (None, 0.9820764489731459)])]

# Упражнения
Упражнения взяты из Rajaraman A., Ullman J. D. Mining of massive datasets. – Cambridge University Press, 2011.


Для выполнения заданий переопределите функции RECORDREADER, MAP, REDUCE. Для модели распределённой системы может потребоваться переопределение функций PARTITION и COMBINER.

### Максимальное значение ряда

Разработайте MapReduce алгоритм, который находит максимальное число входного списка чисел.

In [3]:
import random
from itertools import chain

def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(data, map_func, reduce_func):
    mapped = flatten(map(lambda pair: map_func(*pair), data))
    grouped = group_by_key(mapped)
    return flatten(map(lambda kv: reduce_func(*kv), grouped))

def MAP(key, value):
    yield key, value

def REDUCE(key, values):
    yield max(values)

def RECORDREADER(count):
    return ((None, random.randint(0, 100)) for _ in range(count))

In [6]:
data = list(RECORDREADER(20))
print("Input Data:", [value for _, value in data])

max_value = list(MapReduce(data, MAP, REDUCE))
print("Max Value:", max_value[0])


SyntaxError: invalid syntax (<ipython-input-6-02eb7ba8412d>, line 27)

### Арифметическое среднее

Разработайте MapReduce алгоритм, который находит арифметическое среднее.

$$\overline{X} = \frac{1}{n}\sum_{i=0}^{n} x_i$$


In [7]:
import random
from itertools import chain

def flatten(nested_iterable):
    for iterable in nested_iterable:
        for element in iterable:
            yield element

def groupbykey(iterable):
    t = {}
    for (k2, v2) in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return t.items()

def MapReduce(data, map_func, reduce_func):
    mapped = flatten(map(lambda pair: map_func(*pair), data))
    grouped = groupbykey(mapped)
    return flatten(map(lambda kv: reduce_func(*kv), grouped))

def MAP(key, value):
    yield "sum", value
    yield "count", 1

def REDUCE(key, values):
    yield key, sum(values)

def RECORDREADER(count):
    return ((None, random.randint(0, 100)) for _ in range(count))

In [8]:
data = list(RECORDREADER(20))
print("Input Data:", [value for _, value in data])

results = dict(MapReduce(data, MAP, REDUCE))
mean_value = results["sum"] / results["count"]
print("Mean Value:", mean_value)

Input Data: [31, 47, 57, 89, 92, 30, 41, 82, 7, 34, 24, 80, 13, 63, 68, 11, 0, 32, 63, 66]
Mean Value: 46.5


### GroupByKey на основе сортировки

Реализуйте groupByKey на основе сортировки, проверьте его работу на примерах

In [11]:
import random
from itertools import groupby

def groupByKey_sorted(iterable):
    sorted_iterable = sorted(iterable, key=lambda x: x[0])
    return ((key, [v for _, v in group]) for key, group in groupby(sorted_iterable, key=lambda x: x[0]))

def MAP(key, value):
    yield 1, value

def REDUCE(key, values):
    total = sum(values)
    count = len(values)
    yield total / count

def RECORDREADER(count):
    return ((1, random.randint(0, 100)) for _ in range(count))

def MapReduce(RECORDREADER, MAP, REDUCE):
    return flatten(map(lambda x: REDUCE(*x), groupByKey_sorted(flatten(map(lambda x: MAP(*x), RECORDREADER)))))

In [12]:
array = list(RECORDREADER(100))
print("Input Data:", array)

result = list(MapReduce(array, MAP, REDUCE))
print("Average value:", result[0])

Input Data: [(1, 58), (1, 29), (1, 43), (1, 24), (1, 16), (1, 17), (1, 85), (1, 7), (1, 99), (1, 98), (1, 95), (1, 51), (1, 98), (1, 39), (1, 25), (1, 27), (1, 66), (1, 9), (1, 15), (1, 4), (1, 25), (1, 19), (1, 36), (1, 72), (1, 48), (1, 60), (1, 44), (1, 100), (1, 39), (1, 8), (1, 100), (1, 38), (1, 48), (1, 89), (1, 7), (1, 77), (1, 7), (1, 0), (1, 46), (1, 29), (1, 89), (1, 12), (1, 81), (1, 48), (1, 9), (1, 0), (1, 25), (1, 65), (1, 60), (1, 71), (1, 98), (1, 77), (1, 43), (1, 54), (1, 2), (1, 57), (1, 24), (1, 81), (1, 18), (1, 11), (1, 80), (1, 55), (1, 60), (1, 47), (1, 3), (1, 24), (1, 74), (1, 32), (1, 23), (1, 80), (1, 79), (1, 88), (1, 25), (1, 21), (1, 3), (1, 14), (1, 25), (1, 71), (1, 15), (1, 36), (1, 70), (1, 16), (1, 94), (1, 17), (1, 2), (1, 7), (1, 83), (1, 92), (1, 7), (1, 30), (1, 16), (1, 29), (1, 26), (1, 6), (1, 80), (1, 66), (1, 37), (1, 14), (1, 43), (1, 46)]
Average value: 43.58


### Drop duplicates (set construction, unique elements, distinct)

Реализуйте распределённую операцию исключения дубликатов

In [1]:
import numpy as np
from itertools import chain

# d1 = "it is where it is\nit where it is is what it is"
# d2 = "where is it is is"
# d3 = "it is camera"
d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""

documents = [d1, d2, d3] * 2
maps, reducers = 3, 2

def groupbykey_distributed(map_partitions, PARTITIONER):
    partitions = [{} for _ in range(reducers)]
    for partition in map_partitions:
        for k, v in partition:
            partitions[PARTITIONER(k)].setdefault(k, []).append(v)
    return list(enumerate(map(lambda p: sorted(p.items(), key=lambda x: x[0]), partitions)))

def PARTITIONER(obj):
    return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
    def flatten(lst): return chain.from_iterable(lst)
    def groupbykey(items):
        groups = {}
        for k, v in items:
            groups.setdefault(k, []).append(v)
        return groups.items()

    map_partitions = [list(flatten(MAP(*kv) for kv in reader)) for reader in INPUTFORMAT()]
    if COMBINER:
        map_partitions = [list(flatten(COMBINER(*kv) for kv in groupbykey(partition))) for partition in map_partitions]

    reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER)
    reduce_outputs = [(pid, list(flatten(REDUCE(*group) for group in partition))) for pid, partition in reduce_partitions]

    total_values = sum(len(vs) for _, partition in reduce_partitions for _, vs in partition)
    print(f"{total_values} key-value pairs were sent over a network.")
    return reduce_outputs

def INPUTFORMAT():
    def RECORDREADER(split):
        return ((f"{i}:{j}", line) for i, doc in enumerate(split) for j, line in enumerate(doc.split('\n')))

    split_size = -(-len(documents) // maps)  # Ceiling division
    return (RECORDREADER(documents[i:i+split_size]) for i in range(0, len(documents), split_size))

def MAP(docId, line):
    return ((word, word) for word in line.split())

def REDUCE(key, value):
    yield key

In [2]:
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE)
partitioned_output


50 key-value pairs were sent over a network.


[(0, ['banana', 'it']), (1, ['a', 'is', 'what'])]

#Операторы реляционной алгебры
### Selection (Выборка)

**The Map Function**: Для  каждого кортежа $t \in R$ вычисляется истинность предиката $C$. В случае истины создаётся пара ключ-значение $(t, t)$. В паре ключ и значение одинаковы, равны $t$.

**The Reduce Function:** Роль функции Reduce выполняет функция идентичности, которая возвращает то же значение, что получила на вход.



In [ ]:
import random
from itertools import chain

def MAP(tuple_list):
    return [(t, t) for t in tuple_list if is_even_first_element(t)]

def REDUCE(mapped_items):
    return list(chain.from_iterable(map(lambda x: [x[1]], mapped_items)))

def is_even_first_element(t):
    return t[0] % 2 == 0

def RECORDREADER(count):
    return [tuple(random.randint(0, 100) for _ in range(3)) for _ in range(count)]

def process_data(data, partition_size):
    partitioned_data = [data[i:i + partition_size] for i in range(0, len(data), partition_size)]
    mapped_results = [item for partition in partitioned_data for item in MAP(partition)]
    return REDUCE(mapped_results)

In [ ]:
input_data = RECORDREADER(5)
print("Input Data:", input_data)

result = process_data(input_data, 5)
print("Filter value:", result)

### Projection (Проекция)

Проекция на множество атрибутов $S$.

**The Map Function:** Для каждого кортежа $t \in R$ создайте кортеж $t′$, исключая  из $t$ те значения, атрибуты которых не принадлежат  $S$. Верните пару $(t′, t′)$.

**The Reduce Function:** Для каждого ключа $t′$, созданного любой Map задачей, вы получаете одну или несколько пар $(t′, t′)$. Reduce функция преобразует $(t′, [t′, t′, . . . , t′])$ в $(t′, t′)$, так, что для ключа $t′$ возвращается одна пара  $(t′, t′)$.

In [19]:
from typing import Iterator, NamedTuple

S = {25, 90, 69}

def MAP(t):
    filtered = tuple(el for el in t if el in S)
    return (filtered, filtered)

def REDUCE(key, values: Iterator[NamedTuple]):
    return key, key

def RECORDREADER(count):
    return [(random.randint(0, 100), random.randint(0, 100), random.randint(0, 100)) for _ in range(count)]

def group_by_key(iterable):
    t = {}
    for (k2, v2) in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return t.items()

In [20]:
record = RECORDREADER(5)
print("Input records:", record)

map_output = list(map(MAP, RECORDREADER(100)))
print("MAP output:", map_output)

shuffle_output = group_by_key(map(MAP, RECORDREADER(100)))

reduce_output = list(map(lambda x: REDUCE(*x)[0], list(shuffle_output)))
print("Reduce output:", reduce_output)

Input records: [(75, 22, 96), (12, 62, 44), (86, 29, 66), (54, 24, 96), (5, 62, 85)]
MAP output: [((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((69,), (69,)), ((), ()), ((), ()), ((69,), (69,)), ((90,), (90,)), ((), ()), ((), ()), ((90,), (90,)), ((), ()), ((), ()), ((), ()), ((), ()), ((90,), (90,)), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((90,), (90,)), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((69,), (69,)), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((25,), (25,)), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((69,), (69,)), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ())

### Union (Объединение)

**The Map Function:** Превратите каждый входной кортеж $t$ в пару ключ-значение $(t, t)$.

**The Reduce Function:** С каждым ключом $t$ будет ассоциировано одно или два значений. В обоих случаях создайте $(t, t)$ в качестве выходного значения.

In [5]:
import random
from collections import defaultdict

def MAP(t):
    return t, t

def REDUCE(key, values):
    return key, key

def group_by_key(iterable):
    t = {}
    for (k2, v2) in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return t.items()

def RECORDREADER(count):
    return [(random.randint(0, 100), random.randint(0, 100), random.randint(0, 100)) for _ in range(count)]

In [6]:
record = RECORDREADER(100)
print("Input records:", record)

map_output = map(MAP, record)
print("MAP output:", list(map_output))

shuffle_output = group_by_key(map(MAP, record))
print("Shuffle output:", list(shuffle_output))

reduce_output = [REDUCE(k, v) for k, v in shuffle_output]
print("Reduce output:", reduce_output)

Input records: [(85, 60, 57), (57, 20, 3), (43, 30, 58), (94, 98, 63), (45, 3, 51), (84, 72, 72), (2, 48, 89), (48, 5, 80), (75, 94, 91), (37, 15, 61), (7, 33, 36), (3, 80, 26), (68, 23, 22), (47, 0, 79), (100, 86, 76), (45, 52, 76), (70, 48, 73), (21, 90, 43), (70, 27, 11), (19, 95, 56), (93, 47, 33), (88, 53, 34), (29, 42, 26), (62, 24, 16), (53, 55, 71), (73, 58, 81), (0, 35, 37), (43, 62, 87), (50, 59, 84), (51, 85, 7), (89, 44, 46), (65, 87, 71), (48, 97, 78), (92, 85, 51), (89, 37, 42), (75, 48, 94), (40, 15, 61), (70, 66, 93), (46, 95, 83), (26, 93, 84), (100, 82, 33), (72, 59, 31), (76, 18, 70), (35, 41, 94), (13, 35, 50), (27, 84, 18), (59, 18, 70), (22, 52, 43), (31, 25, 61), (100, 94, 78), (12, 25, 43), (16, 34, 0), (65, 14, 100), (29, 85, 39), (62, 63, 57), (4, 96, 57), (73, 34, 29), (8, 54, 36), (76, 33, 38), (70, 68, 75), (0, 11, 40), (63, 71, 23), (36, 40, 90), (70, 43, 9), (47, 92, 65), (37, 19, 34), (3, 41, 81), (76, 71, 12), (42, 53, 16), (100, 57, 63), (21, 45, 49), 

### Intersection (Пересечение)

**The Map Function:** Превратите каждый кортеж $t$ в пары ключ-значение $(t, t)$.

**The Reduce Function:** Если для ключа $t$ есть список из двух элементов $[t, t]$ $-$ создайте пару $(t, t)$. Иначе, ничего не создавайте.

In [7]:
import random

def MAP(t):
    return t, t

def REDUCE(key, values):
    if len(values) == 2:
        return key, key
    return None

def RECORDREADER(count):
    return [(random.randint(0, 3), random.randint(0, 3)) for _ in range(count)]

def group_by_key(iterable):
    t = {}
    for (k2, v2) in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return t.items()


In [8]:
record = RECORDREADER(100)
print("Input records:", record)

map_output = list(map(MAP, record))
print("MAP output:", map_output)

shuffle_output = list(group_by_key(map_output))
print("Shuffle output:", shuffle_output)

reduce_output = list(filter(None, map(lambda x: REDUCE(*x), shuffle_output)))
print("Reduce output:", reduce_output)

Input records: [(0, 3), (3, 0), (1, 1), (3, 0), (3, 0), (1, 2), (1, 1), (2, 3), (2, 3), (2, 0), (3, 3), (2, 2), (3, 1), (2, 0), (2, 2), (2, 0), (1, 0), (3, 3), (3, 1), (3, 1), (2, 1), (0, 0), (1, 1), (1, 3), (0, 2), (2, 0), (3, 2), (2, 1), (2, 0), (2, 3), (1, 2), (2, 1), (0, 3), (1, 3), (2, 2), (2, 1), (3, 0), (2, 1), (2, 2), (2, 3), (3, 0), (2, 3), (0, 0), (1, 2), (2, 0), (0, 0), (3, 0), (0, 3), (2, 3), (1, 0), (2, 3), (2, 3), (1, 1), (0, 3), (2, 1), (0, 3), (1, 3), (2, 0), (3, 0), (0, 1), (0, 2), (2, 3), (3, 0), (0, 1), (3, 1), (2, 1), (2, 0), (3, 2), (0, 2), (1, 3), (3, 1), (1, 2), (3, 0), (3, 2), (2, 3), (3, 0), (3, 3), (3, 2), (2, 3), (2, 0), (0, 0), (3, 0), (3, 2), (3, 3), (0, 3), (1, 1), (1, 1), (3, 3), (0, 1), (3, 2), (3, 3), (3, 0), (0, 2), (3, 1), (1, 2), (0, 1), (3, 2), (0, 0), (2, 0), (1, 2)]
MAP output: [((0, 3), (0, 3)), ((3, 0), (3, 0)), ((1, 1), (1, 1)), ((3, 0), (3, 0)), ((3, 0), (3, 0)), ((1, 2), (1, 2)), ((1, 1), (1, 1)), ((2, 3), (2, 3)), ((2, 3), (2, 3)), ((2, 0), 

### Difference (Разница)

**The Map Function:** Для кортежа $t \in R$, создайте пару $(t, R)$, и для кортежа $t \in S$, создайте пару $(t, S)$. Задумка заключается в том, чтобы значение пары было именем отношения $R$ or $S$, которому принадлежит кортеж (а лучше, единичный бит, по которому можно два отношения различить $R$ or $S$), а не весь набор атрибутов отношения.

**The Reduce Function:** Для каждого ключа $t$, если соответствующее значение является списком $[R]$, создайте пару $(t, t)$. В иных случаях не предпринимайте действий.

In [13]:
import random

class Tuple:
    def __init__(self, data: tuple, rel_id: int):
        self.data = data
        self.rel_id = rel_id

def get_random_tuple(count):
    data = tuple((random.randint(0, 3), random.randint(0, 3)) for _ in range(count))
    rel_id = random.choice(rels)
    return Tuple(data, rel_id)

def RECORDREADER(count):
    return [get_random_tuple(3) for _ in range(count)]

def MAP(t: Tuple):
    return t.data, t.rel_id

def REDUCE(key, values):
    if rels[0] in values:
        return key, key
    return None

def group_by_key(iterable):
    t = {}
    for (k2, v2) in iterable:
        t.setdefault(k2, []).append(v2)
    return t.items()

In [14]:
rels = [5, 3]
record = RECORDREADER(100)

map_output = [MAP(t) for t in record]
print("MAP output:", map_output)

shuffle_output = list(group_by_key(map_output))
print("Shuffle output:", shuffle_output)

reduce_output = [REDUCE(*x) for x in shuffle_output if REDUCE(*x) is not None]
print("Reduce output:", reduce_output)

MAP output: [(((2, 1), (0, 2), (0, 3)), 5), (((1, 0), (2, 1), (0, 1)), 3), (((2, 1), (3, 2), (1, 1)), 3), (((1, 1), (1, 1), (0, 0)), 3), (((0, 2), (1, 3), (1, 3)), 5), (((3, 0), (2, 3), (1, 0)), 3), (((3, 2), (2, 0), (2, 0)), 5), (((2, 2), (3, 3), (0, 2)), 5), (((0, 3), (3, 2), (3, 3)), 3), (((2, 3), (3, 2), (1, 3)), 5), (((1, 3), (0, 0), (2, 0)), 3), (((2, 2), (2, 2), (0, 1)), 5), (((2, 3), (1, 2), (2, 1)), 3), (((3, 3), (0, 0), (1, 2)), 5), (((0, 2), (2, 1), (3, 3)), 5), (((2, 0), (0, 1), (1, 1)), 5), (((0, 3), (3, 3), (3, 3)), 3), (((0, 2), (2, 2), (1, 1)), 3), (((2, 0), (0, 0), (1, 2)), 5), (((0, 2), (0, 3), (0, 2)), 5), (((1, 3), (1, 1), (1, 1)), 5), (((0, 0), (1, 1), (3, 0)), 3), (((3, 1), (3, 0), (2, 1)), 5), (((0, 1), (2, 1), (0, 1)), 5), (((3, 3), (0, 2), (2, 3)), 5), (((0, 0), (2, 1), (1, 3)), 3), (((2, 1), (1, 3), (1, 0)), 3), (((0, 0), (3, 1), (2, 2)), 5), (((2, 2), (2, 2), (2, 1)), 5), (((3, 2), (3, 3), (3, 2)), 5), (((2, 2), (2, 2), (1, 0)), 3), (((0, 0), (1, 1), (2, 2)),

### Natural Join

**The Map Function:** Для каждого кортежа $(a, b)$ отношения $R$, создайте пару $(b,(R, a))$. Для каждого кортежа $(b, c)$ отношения $S$, создайте пару $(b,(S, c))$.

**The Reduce Function:** Каждый ключ $b$ будет асоциирован со списком пар, которые принимают форму либо $(R, a)$, либо $(S, c)$. Создайте все пары, одни, состоящие из  первого компонента $R$, а другие, из первого компонента $S$, то есть $(R, a)$ и $(S, c)$. На выходе вы получаете последовательность пар ключ-значение из списков ключей и значений. Ключ не нужен. Каждое значение, это тройка $(a, b, c)$ такая, что $(R, a)$ и $(S, c)$ это принадлежат входному списку значений.

In [15]:
import random

rels = [5, 7]

class Tuple:
    def __init__(self, data, rel_id):
        self.data = data
        self.rel_id = rel_id

def generate_random_data(count):
    return [(random.randint(0, 3), random.randint(0, 3)) for _ in range(count)]

def generate_tuple():
    data = generate_random_data(3)
    rel_id = random.choice(rels)
    return Tuple(data, rel_id)

def RECORDREADER(count):
    return [generate_tuple() for _ in range(count)]

def MAP(t: Tuple):
    return (tuple(t.data), t.rel_id)

def REDUCE(key, values):
    if rels[0] in values:
        return key, key
    return None

def group_by_key(iterable):
    grouped = {}
    for key, value in iterable:
        grouped.setdefault(key, []).append(value)
    return grouped.items()

In [16]:
record = RECORDREADER(100)

map_output = []
for t in record:
    result = MAP(t)
    map_output.append(result)
print("MAP output:", map_output)

shuffle_output = group_by_key(map_output)
print("Shuffle output:", shuffle_output)

reduce_output = []
for key, values in shuffle_output:
    result = REDUCE(key, values)
    if result:
        reduce_output.append(result)
print("Reduce output:", reduce_output)

MAP output: [(((0, 0), (0, 0), (2, 1)), 7), (((2, 3), (2, 1), (3, 1)), 7), (((2, 3), (3, 3), (2, 2)), 7), (((1, 3), (3, 0), (1, 2)), 5), (((1, 0), (0, 1), (0, 3)), 7), (((0, 0), (1, 3), (3, 2)), 5), (((1, 2), (1, 1), (1, 3)), 5), (((1, 1), (1, 3), (1, 3)), 7), (((2, 1), (0, 0), (0, 2)), 7), (((2, 2), (3, 1), (2, 3)), 5), (((3, 1), (1, 2), (2, 1)), 5), (((3, 1), (0, 1), (0, 3)), 5), (((2, 2), (3, 0), (3, 3)), 7), (((3, 3), (3, 0), (3, 0)), 7), (((3, 0), (3, 3), (0, 2)), 5), (((3, 2), (1, 2), (0, 0)), 7), (((3, 2), (1, 3), (0, 2)), 5), (((0, 1), (2, 3), (2, 1)), 5), (((1, 3), (3, 1), (2, 0)), 5), (((0, 3), (3, 1), (2, 2)), 5), (((2, 3), (0, 3), (2, 0)), 5), (((2, 3), (2, 1), (3, 2)), 7), (((2, 3), (3, 2), (1, 1)), 5), (((1, 0), (3, 1), (1, 1)), 7), (((0, 2), (0, 1), (3, 2)), 5), (((0, 3), (1, 1), (3, 2)), 7), (((2, 0), (2, 2), (2, 0)), 7), (((3, 3), (3, 0), (1, 2)), 5), (((0, 0), (2, 2), (2, 1)), 7), (((3, 3), (2, 0), (2, 3)), 7), (((1, 3), (1, 3), (2, 0)), 7), (((3, 0), (0, 3), (1, 3)),

### Grouping and Aggregation (Группировка и аггрегация)

**The Map Function:** Для каждого кортежа $(a, b, c$) создайте пару $(a, b)$.

**The Reduce Function:** Ключ представляет ту или иную группу. Примение аггрегирующую операцию $\theta$ к списку значений $[b1, b2, . . . , bn]$ ассоциированных с ключом $a$. Возвращайте в выходной поток $(a, x)$, где $x$ результат применения  $\theta$ к списку. Например, если $\theta$ это $SUM$, тогда $x = b1 + b2 + · · · + bn$, а если $\theta$ is $MAX$, тогда $x$ это максимальное из значений $b1, b2, . . . , bn$.

In [17]:
import random

def get_random_tuple():
    return tuple(random.choices(range(4), k=3))

def RECORDREADER(count):
    return [get_random_tuple() for _ in range(count)]

def MAP(t):
    return t[:2]

def REDUCE(key, values):
    return key, sum(values)

def group_by_key(iterable):
    t = {}
    for (k2, v2) in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return t.items()

In [18]:
record = RECORDREADER(100)

map_output = [MAP(t) for t in record]
print("MAP output:", map_output)

shuffle_output = list(group_by_key(map_output))
print("Shuffle output:", shuffle_output)

reduce_output = [REDUCE(k, v) for k, v in shuffle_output]
print("Reduce output:", reduce_output)

MAP output: [(0, 1), (0, 1), (2, 2), (1, 0), (0, 3), (3, 2), (0, 3), (3, 0), (0, 0), (1, 2), (0, 0), (1, 1), (0, 3), (0, 3), (1, 3), (3, 2), (1, 0), (2, 1), (2, 2), (3, 3), (3, 2), (0, 2), (1, 2), (1, 0), (3, 2), (2, 1), (3, 2), (0, 3), (3, 3), (0, 3), (3, 3), (1, 0), (3, 3), (3, 0), (3, 1), (0, 0), (0, 2), (0, 3), (2, 2), (1, 3), (0, 1), (1, 1), (1, 3), (0, 1), (2, 2), (2, 3), (3, 2), (0, 2), (1, 0), (1, 2), (2, 3), (1, 2), (3, 1), (3, 2), (3, 2), (2, 3), (2, 2), (2, 0), (2, 2), (0, 3), (3, 1), (1, 1), (2, 2), (2, 1), (0, 2), (3, 1), (1, 3), (1, 3), (2, 3), (0, 0), (2, 2), (0, 0), (3, 3), (0, 0), (2, 1), (3, 2), (0, 1), (1, 0), (1, 2), (1, 0), (3, 1), (1, 3), (1, 2), (0, 0), (2, 0), (2, 0), (0, 2), (2, 3), (1, 1), (0, 2), (0, 2), (0, 0), (1, 3), (1, 1), (2, 3), (3, 2), (2, 1), (3, 1), (0, 3), (2, 2)]
Shuffle output: [(0, [1, 1, 3, 3, 0, 0, 3, 3, 2, 3, 3, 0, 2, 3, 1, 1, 2, 3, 2, 0, 0, 0, 1, 0, 2, 2, 2, 0, 3]), (2, [2, 1, 2, 1, 2, 2, 3, 3, 3, 2, 0, 2, 2, 1, 3, 2, 1, 0, 0, 3, 3, 1, 2]), 

#

### Matrix-Vector multiplication

Случай, когда вектор не помещается в памяти Map задачи


## Matrix multiplication (Перемножение матриц)

Если у нас есть матрица $M$ с элементами $m_{ij}$ в строке $i$ и столбце $j$, и матрица $N$ с элементами $n_{jk}$ в строке $j$ и столбце $k$, тогда их произведение $P = MN$ есть матрица $P$ с элементами $p_{ik}$ в строке $i$ и столбце $k$, где

$$p_{ik} =\sum_{j} m_{ij}n_{jk}$$

Необходимым требованием является одинаковое количество столбцов в $M$ и строк в $N$, чтобы операция суммирования по  $j$ была осмысленной. Мы можем размышлять о матрице, как об отношении с тремя атрибутами: номер строки, номер столбца, само значение. Таким образом матрица $M$ предстваляется как отношение $ M(I, J, V )$, с кортежами $(i, j, m_{ij})$, и, аналогично, матрица $N$ представляется как отношение $N(J, K, W)$, с кортежами $(j, k, n_{jk})$. Так как большие матрицы как правило разреженные (большинство значений равно 0), и так как мы можем нулевыми значениями пренебречь (не хранить), такое реляционное представление достаточно эффективно для больших матриц. Однако, возможно, что координаты $i$, $j$, и $k$ неявно закодированы в смещение позиции элемента относительно начала файла, вместо явного хранения. Тогда, функция Map (или Reader) должна быть разработана таким образом, чтобы реконструировать компоненты $I$, $J$, и $K$ кортежей из смещения.

Произведение $MN$ это фактически join, за которым следуют группировка по ключу и аггрегация. Таким образом join отношений $M(I, J, V )$ и $N(J, K, W)$, имеющих общим только атрибут $J$, создаст кортежи $(i, j, k, v, w)$ из каждого кортежа $(i, j, v) \in M$ и кортежа $(j, k, w) \in N$. Такой 5 компонентный кортеж представляет пару элементов матрицы $(m_{ij} , n_{jk})$. Что нам хотелось бы получить на самом деле, это произведение этих элементов, то есть, 4 компонентный кортеж$(i, j, k, v \times w)$, так как он представляет произведение $m_{ij}n_{jk}$. Мы представляем отношение как результат одной MapReduce операции, в которой мы можем произвести группировку и аггрегацию, с $I$ и $K$  атрибутами, по которым идёт группировка, и суммой  $V \times W$.





In [ ]:
# MapReduce model
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

Реализуйте перемножение матриц с использованием модельного кода MapReduce для одной машины в случае, когда одна матрица хранится в памяти, а другая генерируется RECORDREADER-ом.

In [ ]:
import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J) # it is legal to access this from RECORDREADER, MAP, REDUCE
big_mat = np.random.rand(J,K)

def RECORDREADER():
  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      yield ((j,k), big_mat[j,k])

def MAP(k1, v1):
  (j, k) = k1
  w = v1
  # solution code that yield(k2,v2) pairs

def REDUCE(key, values):
  (i, k) = key
  # solution code that yield(k3,v3) pairs

Проверьте своё решение

In [ ]:
# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat)
solution = MapReduce(RECORDREADER, MAP, REDUCE)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution)) # should return true

True

In [ ]:
reduce_output = list(MapReduce(RECORDREADER, MAP, REDUCE))
max(i for ((i,k), vw) in reduce_output)

1

Реализуйте перемножение матриц  с использованием модельного кода MapReduce для одной машины в случае, когда обе матрицы генерируются в RECORDREADER. Например, сначала одна, а потом другая.

In [25]:
import numpy as np

dim1, dim2, dim3 = 5, 8, 20
matrix_a = np.random.rand(dim1, dim2)
matrix_b = np.random.rand(dim2, dim3)

expected_result = np.matmul(matrix_a, matrix_b)

def traverse_nested(iterable):
    for sublist in iterable:
        yield from sublist

def collect_by_key(iterable):
    collection = {}
    for key, value in iterable:
        collection.setdefault(key, []).append(value)
    return collection.items()

def generate_records():
    yield [((0, x, y), matrix_a[x, y]) for x in range(dim1) for y in range(dim2)]
    yield [((1, y, z), matrix_b[y, z]) for y in range(dim2) for z in range(dim3)]

def map_and_join(idx, val):
    matrix_id, x, y = idx
    yield y if matrix_id == 0 else x, (matrix_id, x if matrix_id == 0 else y, val)

def reduce_and_join(key, values):
    part_a = [v for v in values if v[0] == 0]
    part_b = [v for v in values if v[0] == 1]
    for a in part_a:
        for b in part_b:
            yield (a[1], b[1]), a[2] * b[2]

def map_only(idx, val):
    yield idx, val

def reduce_and_sum(key, values):
    yield key, sum(values)

def reconstruct_matrix(reduced_data):
    reduced_data = list(reduced_data)
    rows = max(i for ((i, _), _) in reduced_data) + 1
    cols = max(k for ((_, k), _) in reduced_data) + 1
    result_matrix = np.zeros((rows, cols))
    for (i, k), value in reduced_data:
        result_matrix[i, k] = value
    return result_matrix

def execute_map_reduce(record_function, map_function, reduce_function):
    mapped_data = traverse_nested(map(lambda pair: map_function(*pair), traverse_nested(record_function())))
    reduced_data = traverse_nested(map(lambda pair: reduce_function(*pair), collect_by_key(mapped_data)))
    return reduced_data

In [26]:
intermediate_results = execute_map_reduce(generate_records, map_and_join, reduce_and_join)
final_results = execute_map_reduce(lambda: [intermediate_results], map_only, reduce_and_sum)

assert np.allclose(expected_result, reconstruct_matrix(final_results)), "Ошибка в вычислениях!"
print("MapReduce успешно завершен!")

MapReduce успешно завершен!


Реализуйте перемножение матриц с использованием модельного кода MapReduce Distributed, когда каждая матрица генерируется в своём RECORDREADER.

In [45]:
import numpy as np
from collections import defaultdict

dim1, dim2, dim3 = 5, 8, 20
matrix_a = np.random.rand(dim1, dim2)
matrix_b = np.random.rand(dim2, dim3)
expected_result = np.matmul(matrix_a, matrix_b)

def flatten(nested_iterable):
    for iterable in nested_iterable:
        yield from iterable

def group_by_key(iterable):
    t = {}
    for (k2, v2) in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
    partitions = defaultdict(dict)
    for map_partition in map_partitions:
        for k2, v2 in map_partition:
            partitions[PARTITIONER(k2)][k2] = partitions[PARTITIONER(k2)].get(k2, []) + [v2]
    return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for partition_id, partition in partitions.items()]

def PARTITIONER(obj):
    global reducers
    return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
    map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())

    if COMBINER:
        map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)

    reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER)

    reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

    print(f"{sum(len(vs) for _, vs in flatten([partition for _, partition in reduce_partitions]))} key-value pairs were sent over a network.")
    return reduce_outputs

def asmatrix(reduce_output):
    reduce_output = list(reduce_output)
    I_max = max(i for ((i, _), _) in reduce_output) + 1
    K_max = max(k for ((_, k), _) in reduce_output) + 1
    mat = np.zeros((I_max, K_max))
    for (i, k), value in reduce_output:
        mat[i, k] = value
    return mat

def INPUTFORMAT():
    first_mat = [((0, i, j), val) for i in range(matrix_a.shape[0]) for j in range(matrix_a.shape[1]) for val in [matrix_a[i, j]]]
    second_mat = [((1, j, k), val) for j in range(matrix_b.shape[0]) for k in range(matrix_b.shape[1]) for val in [matrix_b[j, k]]]
    yield first_mat
    yield second_mat

def MAP_JOIN(k1, v1):
    mat_num, i, j = k1
    if mat_num == 0:
        yield j, (mat_num, i, v1)
    else:
        yield i, (mat_num, j, v1)

def REDUCE_JOIN(key, values):
    from_first_mat = [v for v in values if v[0] == 0]
    from_second_mat = [v for v in values if v[0] == 1]
    for f in from_first_mat:
        for s in from_second_mat:
            yield (f[1], s[1]), f[2] * s[2]

def GET_JOINED():
    for j in joined:
        yield j[1]

def MAP_MUL(k1, v1):
    yield k1, v1

def REDUCE_MUL(key, values):
    yield key, sum(values)


partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP_JOIN, REDUCE_JOIN)
joined = [(partition_id, list(partition)) for partition_id, partition in partitioned_output]
print(joined)

mul_output = MapReduceDistributed(GET_JOINED, MAP_MUL, REDUCE_MUL)
pre_result = [(partition_id, list(partition)) for partition_id, partition in mul_output]
print(pre_result)

solution = [v for p in pre_result for v in p[1]]
print(solution)

assert np.allclose(expected_result, asmatrix(solution)), "Результат неверный!"
print("MapReduce успешно выполнен!")

200 key-value pairs were sent over a network.
[(0, [((0, 0), 0.21137974436587442), ((0, 1), 0.05242211928385842), ((0, 2), 0.13003302325742908), ((0, 3), 0.2364188626613045), ((0, 4), 0.050585312546854624), ((0, 5), 0.220939448732282), ((0, 6), 0.11557593468459051), ((0, 7), 0.02025159132721352), ((0, 8), 0.2831917723338421), ((0, 9), 0.25699207914898164), ((0, 10), 0.0307169422559107), ((0, 11), 0.13852709159034146), ((0, 12), 0.27094596854112946), ((0, 13), 0.24834299174787125), ((0, 14), 0.25009370451833696), ((0, 15), 0.0026145072226255095), ((0, 16), 0.010109573831910763), ((0, 17), 0.014948781202788437), ((0, 18), 0.045578012628815424), ((0, 19), 0.1735768515183545), ((1, 0), 0.43240550477317374), ((1, 1), 0.10723644793032416), ((1, 2), 0.2659999198479864), ((1, 3), 0.4836263661574615), ((1, 4), 0.10347901437552648), ((1, 5), 0.4519611571108162), ((1, 6), 0.2364260138872128), ((1, 7), 0.04142733541746832), ((1, 8), 0.579306601164543), ((1, 9), 0.5257116288057307), ((1, 10), 0.062

Обобщите предыдущее решение на случай, когда каждая матрица генерируется несколькими RECORDREADER-ами, и проверьте его работоспособность. Будет ли работать решение, если RECORDREADER-ы будут генерировать случайное подмножество элементов матрицы?

In [47]:
import numpy as np
from collections import defaultdict

dim1, dim2, dim3 = 5, 8, 20
matrix_a = np.random.rand(dim1, dim2)
matrix_b = np.random.rand(dim2, dim3)
expected_result = np.matmul(matrix_a, matrix_b)

def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())

  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)

  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER)

  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output) + 1
  K = max(k for ((i,k), vw) in reduce_output) + 1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

def INPUTFORMAT():
  first_mat = []

  for i in range(matrix_a.shape[0]):
    for j in range(matrix_a.shape[1]):
      first_mat.append(((0, i, j), matrix_a[i,j]))  # первая матрица

  global maps
  split_size = int(np.ceil(len(first_mat)/maps))

  for i in range(0, len(first_mat), split_size):
    yield first_mat[i:i+split_size]

  second_mat = []

  for j in range(matrix_b.shape[0]):
    for k in range(matrix_b.shape[1]):
      second_mat.append(((1, j, k), matrix_b[j,k]))  # вторая матрица

  split_size = int(np.ceil(len(second_mat)/maps))

  for i in range(0, len(second_mat), split_size):
    yield second_mat[i:i+split_size]

def MAP_JOIN(k1, v1):
  (mat_num, i, j) = k1
  w = v1
  if mat_num == 0:
    yield (j, (mat_num, i, w))
  else:
    yield (i, (mat_num, j, w))

def REDUCE_JOIN(key, values):
  from_first_mat = [v for v in values if v[0] == 0]
  from_second_mat = [v for v in values if v[0] == 1]

  for f in from_first_mat:
    for s in from_second_mat:
      yield ((f[1], s[1]), f[2] * s[2])

def GET_JOINED():
  for j in joined:
    print("aa", j)
    yield j[1]

def MAP_MUL(k1, v1):
  yield (k1, v1)

def REDUCE_MUL(key, values):
  res_val = 0

  for v in values:
    res_val += v
  yield (key, res_val)


partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP_JOIN, REDUCE_JOIN, COMBINER=None)
joined = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
print(joined)

mul_output = MapReduceDistributed(GET_JOINED, MAP_MUL, REDUCE_MUL, COMBINER=None)
pre_result = [(partition_id, list(partition)) for (partition_id, partition) in mul_output]
print(pre_result)

solution = [v for p in pre_result for v in p[1]]

print(solution)

assert np.allclose(expected_result, asmatrix(solution)), "Результат неверный!"
print("MapReduce успешно выполнен!")

200 key-value pairs were sent over a network.
[(0, [((0, 0), 0.05556976481605981), ((0, 1), 0.07726493960263937), ((0, 2), 0.1968460389424675), ((0, 3), 0.24793844777423132), ((0, 4), 0.1543138533683553), ((0, 5), 0.06294077292393128), ((0, 6), 0.13180809016579137), ((0, 7), 0.13312377500382663), ((0, 8), 0.06705332711234589), ((0, 9), 0.0745896609599061), ((0, 10), 0.19163448980456269), ((0, 11), 0.022961845532487545), ((0, 12), 0.093365113156367), ((0, 13), 0.0840843577050057), ((0, 14), 0.1621329844242124), ((0, 15), 0.24649754844987745), ((0, 16), 0.18665428226345043), ((0, 17), 0.18214779337907866), ((0, 18), 0.2630201707703219), ((0, 19), 0.1856207226751334), ((1, 0), 0.06359576314259116), ((1, 1), 0.08842439435295191), ((1, 2), 0.22527671494705584), ((1, 3), 0.28374845297230367), ((1, 4), 0.17660156203501295), ((1, 5), 0.07203137353795563), ((1, 6), 0.15084526828946412), ((1, 7), 0.15235097884280102), ((1, 8), 0.07673790816057294), ((1, 9), 0.08536272246236784), ((1, 10), 0.2193